<a href="https://colab.research.google.com/github/LasinduN/torrent-to-google-drive/blob/main/torrentToDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title 1. Mount Google Drive
# Run this cell and authorize access to your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# @title 2. Install libtorrent
# This command installs the necessary library for torrenting.
!pip install libtorrent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 77.1 MB/s eta 0:00:00


In [5]:
# @title 3. Upload .torrent File and Start Download
# Run this cell. It will prompt you to upload your .torrent file.
# After the file is uploaded, the download to your Drive will begin automatically.

from google.colab import files
import libtorrent as lt
import time
import os

# Create a libtorrent session
ses = lt.session()
ses.listen_on(6881, 6891)

# --- Upload the .torrent file ---
print("Please upload your .torrent file:")
uploaded = files.upload()

if not uploaded:
    print("\nNo file was uploaded. Please run the cell again.")
else:
    # Get the path of the uploaded torrent file
    torrent_file_path = list(uploaded.keys())[0]
    print(f"\nUsing torrent file: {torrent_file_path}")

    # Set the save path to a folder in your Google Drive
    # You can change 'Torrents' to any folder name you prefer.
    save_path = '/content/drive/MyDrive/Torrents/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print(f"Created directory: {save_path}")


    # Add the torrent to the session
    info = lt.torrent_info(torrent_file_path)
    params = {
        'save_path': save_path,
        'storage_mode': lt.storage_mode_t.storage_mode_sparse,
        'ti': info
    }
    handle = ses.add_torrent(params)
    ses.start_dht()

    print("\nGot metadata, starting torrent download...")

    # Monitor the download progress
    while (handle.status().state != lt.torrent_status.seeding):
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata', \
                     'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
        print(f'\r{s.progress * 100:.2f}% complete (down: {s.download_rate / 1000:.1f} kB/s up: {s.upload_rate / 1000:.1f} kB/s peers: {s.num_peers}) {state_str[s.state]}', end='')
        time.sleep(1)

    print("\n\nDownload complete and saved to your Google Drive!")

Please upload your .torrent file:


<ipython-input-5-4116470584>:12: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


Saving Game.of.Thrones.SEASON.02.S02.COMPLETE.1080p.10bit.BluRay.6CH.x265.HEVC-PSA-119073.torrent to Game.of.Thrones.SEASON.02.S02.COMPLETE.1080p.10bit.BluRay.6CH.x265.HEVC-PSA-119073 (1).torrent

Using torrent file: Game.of.Thrones.SEASON.02.S02.COMPLETE.1080p.10bit.BluRay.6CH.x265.HEVC-PSA-119073 (1).torrent

Got metadata, starting torrent download...
0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 0) checking fastresume

<ipython-input-5-4116470584>:41: DeprecationWarning: start_dht() is deprecated
  ses.start_dht()


100.00% complete (down: 2215.9 kB/s up: 82.9 kB/s peers: 37) downloading

Download complete and saved to your Google Drive!
